#### 2.1.7 _Programming Exercise:_ Generate segwit v1 addresses for a 2-of-2 MuSig aggregate pubkey

In [ ]:
# Generate individual key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Generate a 2-of-2 aggregate MuSig key
# Method: generate_musig_key(ECPubKey_list)
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Multiply individual keys with challenges
privkey1_c = privkey1 * c_map[pubkey1]
privkey2_c = privkey2 * c_map[pubkey2]
pubkey1_c = pubkey1 * c_map[pubkey1]
pubkey2_c = pubkey2 * c_map[pubkey2]

# Create a segwit v1 address for the MuSig aggregate pubkey
# Method: address = program_to_witness(version_int, program_bytes)
pubkey_data_musig = agg_pubkey.get_bytes()
program_musig = bytes([pubkey_data_musig[0] & 1]) + pubkey_data_musig[1:]
version = 1
address_musig = program_to_witness(version, program_musig)
print("2-of-2 musig: ", address_musig)

#### 2.1.10 _Programming Exercise:_ Create a valid bip-schnorr signature for the MuSig aggregate pubkey

In [ ]:
# Create sighash for ALL (0x00)
sighash_musig = TaprootSignatureHash(spending_tx, [tx.vout[0]], SIGHASH_ALL_TAPROOT, input_index=0)

# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces(nonce_list)
nonce1 = generate_schnorr_nonce()
nonce2 = generate_schnorr_nonce()
R_agg, negated = aggregate_schnorr_nonces([nonce1.get_pubkey(), nonce2.get_pubkey()])

if negated:
    nonce1.negate()
    nonce2.negate()

# Create an aggregate signature
# Method: sign_musig(privkey, nonce, R_agg, agg_pubkey, sighash_musig)
# Method: aggregate_musig_signatures(partial_signature_list, R_agg)
s1 = sign_musig(privkey1_c, nonce1, R_agg, agg_pubkey, sighash_musig)
s2 = sign_musig(privkey2_c, nonce2, R_agg, agg_pubkey, sighash_musig)
sig_agg = aggregate_musig_signatures([s1, s2], R_agg)
print("Aggregate signature is {}\n".format(sig_agg.hex()))

# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(sig_agg)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")